In [1]:
import torch
print(torch.__version__)

2.0.0+cu117


### Register datasets

In [2]:
from detectron2.utils.logger import setup_logger
logger = setup_logger()

In [3]:
import matplotlib.pyplot as plt
#%matplotlib notebook
def imgshow(title="", image = None, size = 6):
    w, h = image.shape[0], image.shape[1]
    aspect_ratio = w/h
    plt.figure(figsize=(size * aspect_ratio,size))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.show()
    
def cv2_imshow(im):
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    plt.figure(), plt.imshow(im), plt.axis('off')

In [4]:
# Some configurations
#@title Register dataset
from detectron2.data.datasets import register_coco_instances
import os
cur_dir = os.getcwd()
# dataset path




#data_dir = r"D:\Nyi Zaw Aung\815_CowDataChecking\Sumiyoshi ToAnnotate\May 2025\Combine"
data_dir = r"D:\Nyi Zaw Aung\815_CowDataChecking\Sumiyoshi ToAnnotate\September 2025"

training_dataset_name = "testing_data"
training_json_file = os.path.join(data_dir, "training_images\\0_SideLane_addedversion_Training_COCO.json")
training_img_dir = os.path.join(data_dir, "training_images")
register_coco_instances(training_dataset_name, {}, training_json_file, training_img_dir)

test_dataset_name = "test_data"
test_json_file =  os.path.join(data_dir, "testing_images\\0_SideLane_addedversion_Testing_COCO.json")
test_img_dir =  os.path.join(data_dir, "testing_images")
register_coco_instances(test_dataset_name, {}, test_json_file, test_img_dir)

### Training configuration

In [5]:
import os
from detectron2.config import get_cfg
from detectron2 import model_zoo
import pickle

In [6]:
output_dir = f"{data_dir}\\detection_Tanaka_weights_with_initial"
os.makedirs(output_dir, exist_ok=True)
nc = 3
device = "cuda"
# Select a model
config_file_url = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
checkpoint_url = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
#from detectron2.projects import point_rend
#config_file = "detectron2/projects/PointRend/configs/InstanceSegmentation/pointrend_rcnn_X_101_32x8d_FPN_3x_coco.yaml"
#checkpoint_file = "detectron2://PointRend/InstanceSegmentation/pointrend_rcnn_X_101_32x8d_FPN_3x_coco/28119989/model_final_ba17b9.pkl"# config_file = "detectron2/projects/PointRend/configs/InstanceSegmentation/pointrend_rcnn_R_50_FPN_3x_coco.yaml"

In [7]:
# Create a configuration file
cfg = get_cfg()
config_file = model_zoo.get_config_file(config_file_url)
cfg.merge_from_file(config_file)
# # # Download weights
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(checkpoint_url)
# Create a configuration and set up the model and datasets
# model_file = "F:\\Mounting_Model_Data\\Yoshii\\maskrcnn_WH_Feb22_Yoshii"
# cfg = get_cfg()
# # loading initial configuration file (can change here with above model file or previous dataset weight to update)
# cfg.merge_from_file(f"{model_file}\\config.yml")
# # loading initial weight file
# cfg.MODEL.WEIGHTS = f"{model_file}\\model_final.pth"
#from detectron2.projects import point_rend
# Select a lighter model to train faster
#config_file = "detectron2/projects/PointRend/configs/InstanceSegmentation/pointrend_rcnn_R_50_FPN_3x_coco.yaml"
#checkpoint_file = "detectron2://PointRend/InstanceSegmentation/pointrend_rcnn_R_50_FPN_3x_coco/164955410/model_final_edd263.pkl"
#config_file = "detectron2/projects/PointRend/configs/InstanceSegmentation/pointrend_rcnn_X_101_32x8d_FPN_3x_coco.yaml"
#checkpoint_file = "detectron2://PointRend/InstanceSegmentation/pointrend_rcnn_X_101_32x8d_FPN_3x_coco/28119989/model_final_ba17b9.pkl"
# Create a configuration file
#cfg = get_cfg()
# Add PointRend-specific config
#point_rend.add_pointrend_config(cfg)
#cfg.merge_from_file(config_file)
#cfg = get_cfg()
# Create a configuration and set up the model and datasets
# model_file = "F:\\Mounting_Model_Data\\Tanaka\\camera_comparasion\\new_camera\\version1\\version6\\Tanaka_roi_80_model_6_with_background_with_oversampled"
# # loading initial configuration file (can change here with above model file or previous dataset weight to update)
# cfg.merge_from_file(f"{model_file}\\config.yml")
# # loading initial weight file
# cfg.MODEL.WEIGHTS = f"{model_file}\\model_best.pth"


# Set datasets
cfg.DATASETS.TRAIN = (training_dataset_name,)

#cfg.INPUT.MIN_SIZE_TRAIN = (2192,)
#cfg.INPUT.MAX_SIZE_TRAIN = 2992

# Workers
cfg.DATALOADER.NUM_WORKERS = 4
# Options: TrainingSampler, RepeatFactorTrainingSampler
cfg.DATALOADER.SAMPLER_TRAIN = "TrainingSampler"
# Repeat threshold for RepeatFactorTrainingSampler
cfg.DATALOADER.REPEAT_THRESHOLD = 0.3
# Images per batch
cfg.SOLVER.IMS_PER_BATCH = 16
# Learning rate
cfg.SOLVER.BASE_LR = 0.0005
cfg.SOLVER.STEPS = (10000, 13000)
# Iterations
cfg.SOLVER.MAX_ITER = 15000
cfg.SOLVER.CHECKPOINT_PERIOD = 1000
# NMS threshold used on RPN proposals
cfg.MODEL.RPN.NMS_THRESH = 0.5
# Evaluation
cfg.TEST.EVAL_PERIOD = cfg.SOLVER.CHECKPOINT_PERIOD
# Classes
cfg.MODEL.ROI_HEADS.NUM_CLASSES = nc
cfg.MODEL.DEVICE = device
cfg.OUTPUT_DIR = output_dir
# New
#cfg.MODEL.POINT_HEAD.NUM_CLASSES = nc

### A trainer

In [8]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
import detectron2.data.transforms as T
from detectron2.data import DatasetMapper
from detectron2.data import build_detection_train_loader
class MyTrainer(DefaultTrainer):
  @classmethod
  def build_train_loader(cls, cfg):
    #def build_evaluator(cls, cfg, training_dataset_name, output_folder=None):
        # Set up data augmentation
        augs = [T.ResizeShortestEdge(
                    [500,640, 672, 704, 736, 768, 800],
                    max_size=1333, sample_style="choice"),
                T.RandomBrightness(0.5, 1.5),
                T.RandomSaturation(0.5, 1.5),
                T.RandomFlip(prob=0.5,horizontal=True, vertical = False),
                #T.RandomFlip(prob=0.5,horizontal=False, vertical = True),
                #T.RandomRotation(angle = [-90, 90], sample_style  = "range", center = [[0.4, 0.6], [0.4, 0.6]], expand = False),
#                 #transforms.MinIoURandomCrop(min_ious=(0.1, 0.3, 0.5, 0.7, 0.9), min_crop_size=0.3, mode_trials=1000, crop_trials=50)
                ]
                #transforms.RandomFlip(prob=0.5)
        data_loader = build_detection_train_loader(cfg,
            mapper=DatasetMapper(cfg, is_train=True, 
                                 augmentations=augs))
        
        #return build_detection_train_loader(cfg, mapper=custom_mapper) 
        return data_loader
  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):
    if output_folder == None:
      output_folder = cfg.OUTPUT_DIR
    else:
      output_folder = os.path.join(cfg.OUTPUT_DIR, output_folder)
      os.makedirs(output_folder)
    # Use 
    return COCOEvaluator(dataset_name, distributed=False, output_dir=output_folder)

In [9]:
trainer = MyTrainer(cfg)

[09/26 13:19:01 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

### Preprocessing Input Images

In [10]:
print("MIN_SIZE_TRAIN", cfg.INPUT.MIN_SIZE_TRAIN)
print("MAX_SIZE_TRAIN", cfg.INPUT.MAX_SIZE_TRAIN)

MIN_SIZE_TRAIN (640, 672, 704, 736, 768, 800)
MAX_SIZE_TRAIN 1333


In [11]:
trainer._data_loader_iter = iter(trainer.data_loader)
data = next(trainer._data_loader_iter)
#or i in data:
#  print(i['image'].shape)

In [12]:
images = trainer.model.preprocess_image(data)
#print(images.tensor.shape)

In [13]:
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt
# from detectron2.utils.visualizer import Visualizer

# for i, item in enumerate(images.tensor[:3]):
#   img = np.moveaxis(item.to("cpu").numpy(), 0, -1)
#   pixel_mean = cfg.MODEL.PIXEL_MEAN
#   pixel_std = cfg.MODEL.PIXEL_STD
#   img = (img * pixel_std) + pixel_mean
#   v = Visualizer(img, metadata={}, scale=0.5)
#   v = v.overlay_instances(boxes=data[i]['instances'].get('gt_boxes'),)
#   dpi = 80
#   im_data = v.get_image()[:,:, ::-1]
#   height, width, depth = im_data.shape
#   figsize = width / float(dpi), height / float(dpi)
#   fig = plt.figure(figsize=figsize)
#   plt.imshow(im_data)
#   plt.imshow(im_data)
#   plt.axis("off")
#   plt.show()

### Sampling training data and generating the default anchors

In [14]:
from tqdm import tqdm
import torch
def get_gt_boxes_batch(data):
  gt_boxes = [
      item['instances'].get('gt_boxes').tensor 
      for item in data
      ]
  return torch.cat(gt_boxes)

def get_gt_boxes(trainer, iterations):
  trainer._data_loader_iter = iter(trainer.data_loader)
  gt_boxes = [
      get_gt_boxes_batch(next(trainer._data_loader_iter)) 
      for _ in tqdm(range(iterations))
      ]
  return torch.cat(gt_boxes)
  
gt_boxes = get_gt_boxes(trainer, 1000)
print()
print(gt_boxes.shape)

100%|██████████| 1000/1000 [05:41<00:00,  2.93it/s]


torch.Size([80933, 4])


In [15]:
def boxes2wh(boxes):
  x1y1 = boxes[:, :2]
  x2y2 = boxes[:, 2:]
  return x2y2 - x1y1

gt_wh = boxes2wh(gt_boxes)
print(gt_wh.min(), gt_wh.max())

tensor(17.8318) tensor(493.3333)


In [16]:
print("sizes", cfg.MODEL.ANCHOR_GENERATOR.SIZES)
print("ratios", cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS)

sizes [[32], [64], [128], [256], [512]]
ratios [[0.5, 1.0, 2.0]]


In [17]:
generate_anchors = trainer.model.proposal_generator.anchor_generator.generate_cell_anchors
anchors = generate_anchors()
ac_wh = boxes2wh(anchors)

### Generating sizes and ratios hyperparameters

In [18]:
def wh2size(wh):
  return torch.sqrt(gt_wh[:, 0]*gt_wh[:, 1])

def wh2ratio(wh):
  return wh[:, 1]/wh[:,0]

gt_sizes = wh2size(gt_wh)
gt_ratios = wh2ratio(gt_wh)

In [19]:
def best_ratio(ac_wh, gt_wh):
  all_ratios = gt_wh[:, None] / ac_wh[None]
  inverse_ratios = 1/all_ratios
  ratios = torch.min(
      all_ratios, inverse_ratios
  )
  worst = ratios.min(-1).values
  best = worst.max(-1).values
  return best

In [20]:
def fitness(ac_wh, gt_wh, EDGE_RATIO_THRESHOLD = 0.25):
  ratio = best_ratio(ac_wh, gt_wh)
  return (ratio * (ratio > EDGE_RATIO_THRESHOLD).float()).mean()

In [21]:
def best_recall(ac_wh, gt_wh, EDGE_RATIO_THRESHOLD = 0.25):
  ratio = best_ratio(ac_wh, gt_wh)
  best = (ratio > EDGE_RATIO_THRESHOLD).float().mean()
  return best

In [22]:
print("recall", best_recall(ac_wh, gt_wh))
print("fitness", fitness(ac_wh, gt_wh))

recall tensor(1.)
fitness tensor(0.7526)


In [23]:
import numpy as np
import pandas as pd
#import 
#plt.rcParams["figure.dpi"] = 150
from scipy.cluster.vq import kmeans
def estimate_clusters(values, num_clusters, iter=100):
  std = values.std(0).item()
  k, _ = kmeans(values / std, num_clusters, iter=iter)
  k *= std
  return k

def visualize_clusters(values, centers):
  plt.hist(values, histtype='step')
  plt.scatter(centers, [0]*len(centers), c= "red")
  plt.show()

In [24]:
sizes = estimate_clusters(gt_sizes, 5)
#visualize_clusters(gt_sizes, sizes)
ratios = estimate_clusters(gt_ratios, 3)
#visualize_clusters(gt_ratios, ratios)

In [25]:
fitness(boxes2wh(generate_anchors(sizes, ratios)), gt_wh)
print("sizes", sizes)
print("ratios", ratios)

sizes [154.52798   86.867226 187.08473  121.45394  233.81372 ]
ratios [2.155631   1.0543213  0.52571815]


### Evolve the results using the Genetic Algorithm

In [26]:
def evolve(sizes, ratios, 
           gt_wh, 
           iterations=10000, 
           probability=0.9, 
           muy=1, 
           sigma=0.05, 
           fit_fn=fitness, 
           verbose=False):
  anchors = generate_anchors(tuple(sizes), tuple(ratios))
  ac_wh = boxes2wh(anchors)
  best_fit = fit_fn(ac_wh, gt_wh)
  anchor_shape = len(sizes) + len(ratios)

  pbar = tqdm(range(iterations), desc=f"Evolving ratios and sizes:")
  for i, _ in enumerate(pbar):
      # to mutate and how much
      mutation = np.ones(anchor_shape)
      mutate = np.random.random(anchor_shape) < probability
      mutation = np.random.normal(muy, sigma, anchor_shape)*mutate
      mutation = mutation.clip(0.3, 3.0)
      # mutated
      mutated_sizes = sizes.copy()*mutation[:len(sizes)]
      mutated_ratios = ratios.copy()*mutation[-len(ratios):]
      mutated_anchors = generate_anchors(
          tuple(mutated_sizes), 
          tuple(mutated_ratios))
      mutated_ac_wh = boxes2wh(mutated_anchors)
      mutated_fit = fit_fn(mutated_ac_wh, gt_wh)

      if mutated_fit > best_fit:
        sizes = mutated_sizes.copy()
        ratios = mutated_ratios.copy()
        best_fit = mutated_fit
        pbar.desc = (f"Evolving ratios and sizes, Fitness = {best_fit:.4f}")
  return sizes, ratios

In [27]:
e_sizes, e_ratios = evolve(sizes, ratios, gt_wh, verbose=False)

Evolving ratios and sizes, Fitness = 0.8646: 100%|██████████| 10000/10000 [02:16<00:00, 73.29it/s]


In [28]:
print("sizes", e_sizes)
print("ratios", e_ratios)

sizes [149.95695869  90.25478231 179.26082761 119.37182681 213.92048847]
ratios [1.36056301 0.7196418  0.44608757]


In [29]:
#separate array with ", " for each element
e_sizes = np.array(e_sizes)
e_ratios = np.array(e_ratios)
e_sizes_str = np.array2string(e_sizes, separator=", ", formatter={'float_kind': lambda x: "%.8f" % x})
e_ratios_str = np.array2string(e_ratios, separator=", ", formatter={'float_kind': lambda x: "%.8f" % x})
print("sizes", e_sizes_str)   
print("ratios", e_ratios_str)

sizes [149.95695869, 90.25478231, 179.26082761, 119.37182681, 213.92048847]
ratios [1.36056301, 0.71964180, 0.44608757]
